<a href="https://colab.research.google.com/github/harshithakolipaka/Million_Song_Recommendation_Engine/blob/main/SONG_RECOMMENDATION_SYSTEM_PROJECT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# User Defined Recommender Module

In [ ]:
import numpy as np
import pandas
#Class for Item similarity based Recommender System model
class item_similarity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.cooccurence_matrix = None
        self.songs_dict = None
        self.rev_songs_dict = None
        self.item_similarity_recommendations = None
        
    #Get unique items (songs) corresponding to a given user
    def get_user_items(self, user):
        user_data = self.train_data[self.train_data[self.user_id] == user]
        user_items = list(user_data[self.item_id].unique())
        
        return user_items
        
    #Get unique users for a given item (song)
    def get_item_users(self, item):
        item_data = self.train_data[self.train_data[self.item_id] == item]
        item_users = set(item_data[self.user_id].unique())
            
        return item_users
        
    #Get unique items (songs) in the training data
    def get_all_items_train_data(self):
        all_items = list(self.train_data[self.item_id].unique())
            
        return all_items
        
    #Construct cooccurence matrix
    def construct_cooccurence_matrix(self, user_songs, all_songs):
            
        ####################################
        #Get users for all songs in user_songs.
        ####################################
        user_songs_users = []        
        for i in range(0, len(user_songs)):
            user_songs_users.append(self.get_item_users(user_songs[i]))
            
        ###############################################
        #Initialize the item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        ###############################################
        cooccurence_matrix = np.matrix(np.zeros(shape=(len(user_songs), len(all_songs))), float)
           
        #############################################################
        #Calculate similarity between user songs and all unique songs
        #in the training data
        #############################################################
        for i in range(0,len(all_songs)):
            #Calculate unique listeners (users) of song (item) i
            songs_i_data = self.train_data[self.train_data[self.item_id] == all_songs[i]]
            users_i = set(songs_i_data[self.user_id].unique())
            
            for j in range(0,len(user_songs)):       
                    
                #Get unique listeners (users) of song (item) j
                users_j = user_songs_users[j]
                    
                #Calculate intersection of listeners of songs i and j
                users_intersection = users_i.intersection(users_j)
                
                #Calculate cooccurence_matrix[i,j] as Jaccard Index
                if len(users_intersection) != 0:
                    #Calculate union of listeners of songs i and j
                    users_union = users_i.union(users_j)
                    
                    cooccurence_matrix[j,i] = float(len(users_intersection))/float(len(users_union))
                else:
                    cooccurence_matrix[j,i] = 0
                    
        
        return cooccurence_matrix

    
    #Use the cooccurence matrix to make top recommendations
    def generate_top_recommendations(self, user, cooccurence_matrix, all_songs, user_songs):
        print("Non zero values in cooccurence_matrix :%d" % np.count_nonzero(cooccurence_matrix))
        
        #Calculate a weighted average of the scores in cooccurence matrix for all user songs.
        user_sim_scores = cooccurence_matrix.sum(axis=0)/float(cooccurence_matrix.shape[0])
        user_sim_scores = np.array(user_sim_scores)[0].tolist()
 
        #Sort the indices of user_sim_scores based upon their value
        #Also maintain the corresponding score
        sort_index = sorted(((e,i) for i,e in enumerate(list(user_sim_scores))), reverse=True)
    
        #Create a dataframe from the following
        columns = ['user_id', 'song', 'score', 'rank']
        #index = np.arange(1) # array of numbers for the number of samples
        df = pandas.DataFrame(columns=columns)
         
        #Fill the dataframe with top 10 item based recommendations
        rank = 1 
        for i in range(0,len(sort_index)):
            if ~np.isnan(sort_index[i][0]) and all_songs[sort_index[i][1]] not in user_songs and rank <= 10:
                df.loc[len(df)]=[user,all_songs[sort_index[i][1]],sort_index[i][0],rank]
                rank = rank+1
        
        #Handle the case where there are no recommendations
        if df.shape[0] == 0:
            print("The current user has no songs for training the item similarity based recommendation model.")
            return -1
        else:
            return df
 
    #Create the item similarity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

    #Use the item similarity based recommender system model to
    #make recommendations
    def recommend(self, user):
        
        ########################################
        #A. Get all unique songs for this user
        ########################################
        user_songs = self.get_user_items(user)    
            
        print("No. of unique songs for the user: %d" % len(user_songs))
        
        ######################################################
        #B. Get all unique items (songs) in the training data
        ######################################################
        all_songs = self.get_all_items_train_data()
        
        print("no. of unique songs in the training set: %d" % len(all_songs))
         
        ###############################################
        #C. Construct item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        ###############################################
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        
        #######################################################
        #D. Use the cooccurence matrix to make recommendations
        #######################################################
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)
                
        return df_recommendations
    
    #Get similar items to given items
    def get_similar_items(self, item_list):
        
        user_songs = item_list
        
        ######################################################
        #B. Get all unique items (songs) in the training data
        ######################################################
        all_songs = self.get_all_items_train_data()
        
        print("no. of unique songs in the training set: %d" % len(all_songs))
         
        ###############################################
        #C. Construct item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        ###############################################
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        
        #######################################################
        #D. Use the cooccurence matrix to make recommendations
        #######################################################
        user = ""
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)
         
        return df_recommendations

# Connecting Files

In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


# Import Modules

In [ ]:
import pandas as pd
import numpy as np

# Loading Dataset

In [ ]:
#triplets_file
song_df_1 = pd.read_csv("/content/drive/MyDrive/SONG RECOMMENDATION DATASET/triplets_file.csv")
song_df_1.head(5)

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [ ]:
#song_data_file
song_df_2 = pd.read_csv("/content/drive/MyDrive/SONG RECOMMENDATION DATASET/song_data.csv")
song_df_2.head(5)

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [ ]:
print(len(song_df_1), len(song_df_2))

2000000 1000000


# Pre-Processing Data

In [ ]:
song_df = pd.merge(song_df_1, song_df_2.drop_duplicates(["song_id"]), on = "song_id", how = "left")
song_df.head(5)

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [ ]:
len(song_df)

2000000

In [ ]:
song_df["song"] = song_df["title"] + " - " + song_df["artist_name"]
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0,The Cove - Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005,Constellations - Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999,Learn To Fly - Foo Fighters


In [ ]:
#cummulative sum of listen_count
song_grouped = song_df.groupby(["song"]).agg({"listen_count" : "count"}).reset_index()
song_grouped.head()

,song,listen_count
0,& Down - Boys Noize,1
1,(Nice Dream) - Radiohead,1
2,(You Drive Me) Crazy (The Stop Remix!) - Britn...,1
3,16 Candles - The Crests,1
4,1983 (Daedelus's Odd-Dance Party Remix) - Flyi...,1


In [ ]:
len(song_grouped)

896

# Popularity Recommendation 


In [ ]:
#calculating percentage for each song
grouped_sum = song_df["listen_count"].sum()
song_grouped["percentage"] = (song_grouped["listen_count"] / grouped_sum ) * 100
song_grouped = song_grouped.sort_values(["listen_count", "song"], ascending = [0,1])
song_grouped.reset_index().head()

,index,song,listen_count,percentage
0,604,Sehr kosmisch - Harmonia,4,0.149254
1,22,Ain't No Rest For The Wicked (Original Version...,3,0.111940
2,132,Clocks - Coldplay,3,0.111940
3,223,Fireflies - Charttraxx Karaoke,3,0.111940
4,225,Float On - Modest Mouse,3,0.111940


In [ ]:
#calculating rank 
song_grouped["rank"] = song_grouped["listen_count"].rank(ascending = 0, method = "first") 
song_grouped.drop(["percentage"], axis = 1).head().sort_values(["rank"])

,song,listen_count,rank
604,Sehr kosmisch - Harmonia,4,1.0
22,Ain't No Rest For The Wicked (Original Version...,3,2.0
132,Clocks - Coldplay,3,3.0
223,Fireflies - Charttraxx Karaoke,3,4.0
225,Float On - Modest Mouse,3,5.0


# Item Similarity Recommendation

In [ ]:
song_df = song_df.head(10000)

In [ ]:
ir = item_similarity_recommender_py()
ir.create(song_df, "user_id", "song")

In [ ]:
ir.recommend(song_df["user_id"][5])

No. of unique songs for the user: 45
no. of unique songs in the training set: 896
Non zero values in cooccurence_matrix :2305


,user_id,song,score,rank
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Angie (1993 Digital Remaster) - The Rolling St...,0.011852,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,High Life - Daft Punk,0.011852,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Scythian Empires - Andrew Bird,0.011111,3
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Dark Matter - Andrew Bird,0.011111,4
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,You Don't Know Me (featuring Regina Spektor) -...,0.011111,5
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,These Walls Are Thin (Album Version) - The Box...,0.011111,6
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,I Never (Album Version) - Rilo Kiley,0.011111,7
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Under The Sheets - Ellie Goulding,0.011111,8
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Underwater World - ATB,0.011111,9
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Freeze - K-OS,0.011111,10


In [ ]:
ir.get_similar_items(["Dark Matter - Andrew Bird"])

no. of unique songs in the training set: 896
Non zero values in cooccurence_matrix :100


,user_id,song,score,rank
0,,Scythian Empires - Andrew Bird,1.0,1
1,,You Don't Know Me (featuring Regina Spektor) -...,1.0,2
2,,These Walls Are Thin (Album Version) - The Box...,1.0,3
3,,I Never (Album Version) - Rilo Kiley,1.0,4
4,,Under The Sheets - Ellie Goulding,1.0,5
5,,Underwater World - ATB,1.0,6
6,,Freeze - K-OS,1.0,7
7,,Road To Nowhere (Remastered LP Version ) - Tal...,1.0,8
8,,The Good That Won't Come Out - Rilo Kiley,1.0,9
9,,Phantom Limb (Album) - The Shins,1.0,10
